In [14]:

#in this file, we execute the rank for stations and do subroutes divide 
import arcpy
import datetime
from collections import defaultdict
from tqdm.auto import tqdm

In [15]:
from itertools import groupby

In [16]:
# Assuming 'stations_featureclass' has 'route_id', 'stream_order', and 'station_id'
# Set the workspace
arcpy.env.workspace = "D:/MyApps/ArcGIS Pro 3.2/greenbelt_gis/Input/MyProject1.gdb"

stations_fc = 'D:/MyApps/ArcGIS Pro 3.2/greenbelt_gis/Input/MyProject1.gdb/SW_stations'
routes_fc = "D:/MyApps/ArcGIS Pro 3.2/greenbelt_gis/Input/MyProject1.gdb/forlooptraced_routes_SW_ExportFeatures"
subroutes_fc = 'subroutes_featureclass_SW'

In [17]:


# Ensure the subroutes feature class exists
arcpy.CreateFeatureclass_management(
    out_path=arcpy.env.workspace,
    out_name="subroutes_featureclass_SW",
    geometry_type="POLYLINE",
    spatial_reference=arcpy.Describe(routes_fc).spatialReference)

# Output feature class for the sub-routes
subroutes_fc = 'subroutes_featureclass_SW'

route_field_names = [field.name for field in arcpy.ListFields(subroutes_fc)]

# Check if 'route_id' and 'route_rank' exist, create them if not
if 'route_id' not in route_field_names:
    arcpy.AddField_management(subroutes_fc, 'route_id', 'TEXT')
if 'route_rank' not in route_field_names:
    arcpy.AddField_management(subroutes_fc, 'route_rank', 'LONG')

In [18]:
# d = dict()

# for i in range(10):
#     key = str(i)
#     d[i] = dict()
#     for j in range(10):
#         d[i][j] = 0

In [6]:
station_route_ranks = defaultdict(lambda: defaultdict(int))
print (station_route_ranks)



defaultdict(<function <lambda> at 0x000002801E09AAF0>, {})


In [8]:
# out put should be a  nested dictionary = {{station_id: {route_id{route_rank}, }}}
def assign_route_ranks_to_stations(stations_fc, routes_fc):
    # Create a dictionary to hold station stream orders
    station_stream_orders = {}
    with arcpy.da.SearchCursor(stations_fc, ["STATION", "shreve"]) as cursor:
        for station_id, stream_order in cursor:
            # print(station_id, end=' ')

            station_stream_orders[station_id] = stream_order

    # Process each route and assign ranks to stations based on stream order
    with arcpy.da.SearchCursor(routes_fc, ["route_id", "station_list_field"]) as cursor:
        for route_id, station_ids in cursor:
            print(station_ids, end=' ')
            # Convert the stored string of station_ids to a list and sort by stream order
            sorted_station_ids = sorted(station_ids.split(','), key=lambda id: station_stream_orders.get(id, float('inf')))
            
            # Assign rank for each station in the route
            for rank, station_id in enumerate(sorted_station_ids, start=1):
                station_route_ranks[station_id][route_id] = rank
            
# Call the function
assign_route_ranks_to_stations(stations_fc,routes_fc)

# Example: print the station_route_ranks dictionary
for station_id, routes in tqdm(station_route_ranks.items()):
    print(f"Station ID: {station_id}")
    for route_id, rank in routes.items():
        print(f" - Route ID: {route_id}, Rank: {rank}")
     

3007703502.0,3007704002.0,3007701302.0,3007702202.0,3007702302.0,3007702702.0 3007703602.0,3007701302.0,3007702202.0,3007702302.0 3007703802.0,3007700802.0,3007701302.0,3007702202.0,3007702302.0 3007703902.0,3007700102.0,3007701302.0,3007702202.0,3007702302.0 3007704002.0,3007701302.0,3007702202.0,3007702302.0,3007702702.0 3007704102.0,3007701002.0,3007701302.0,3007702202.0,3007702302.0,3007702502.0 3008500102.0 3008500102.0,3008500202.0,3008500302.0,3008501102.0 3008500102.0,3008500302.0 3008500102.0,3008500302.0,3008500402.0 3008500102.0,3008500302.0,3008500502.0,3008501202.0,3008501302.0,3008501502.0 3008500102.0,3008500302.0,3008500502.0,3008500602.0,3008501202.0,3008501302.0,3008501502.0 3008500102.0,3008500302.0,3008500502.0,3008500602.0,3008500702.0,3008501202.0,3008501302.0,3008501502.0 3008500102.0,3008500302.0,3008500502.0,3008500602.0,3008500702.0,3008500802.0,3008501202.0,3008501302.0,3008501502.0 3008500102.0,3008500302.0,3008500402.0,3008500902.0 3008500102.0,3008500202.0

  0%|          | 0/1274 [00:00<?, ?it/s]

Station ID: 3007703502.0
 - Route ID: 1, Rank: 1
 - Route ID: 32, Rank: 1
 - Route ID: 134, Rank: 1
 - Route ID: 139, Rank: 1
Station ID: 3007704002.0
 - Route ID: 1, Rank: 2
 - Route ID: 5, Rank: 1
 - Route ID: 32, Rank: 2
 - Route ID: 134, Rank: 2
 - Route ID: 139, Rank: 2
Station ID: 3007701302.0
 - Route ID: 1, Rank: 3
 - Route ID: 2, Rank: 2
 - Route ID: 3, Rank: 3
 - Route ID: 4, Rank: 3
 - Route ID: 5, Rank: 2
 - Route ID: 6, Rank: 3
 - Route ID: 29, Rank: 3
 - Route ID: 30, Rank: 2
 - Route ID: 31, Rank: 2
 - Route ID: 32, Rank: 5
 - Route ID: 33, Rank: 3
 - Route ID: 34, Rank: 2
 - Route ID: 126, Rank: 2
 - Route ID: 127, Rank: 2
 - Route ID: 128, Rank: 4
 - Route ID: 129, Rank: 4
 - Route ID: 130, Rank: 3
 - Route ID: 131, Rank: 5
 - Route ID: 132, Rank: 6
 - Route ID: 133, Rank: 2
 - Route ID: 134, Rank: 4
 - Route ID: 135, Rank: 2
 - Route ID: 136, Rank: 4
 - Route ID: 137, Rank: 2
 - Route ID: 138, Rank: 1
 - Route ID: 139, Rank: 3
 - Route ID: 140, Rank: 2
 - Route ID: 14

In [9]:
#export to a csv file
import csv
with open('station_route_ranks.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['station_id', 'route_id', 'route_rank'])
    for station_id, routes in station_route_ranks.items():
        for route_id, rank in routes.items():
            writer.writerow([station_id, route_id, rank])

In [13]:
def create_subroutes_using_sorted_stations(routes_fc, stations_fc, sorted_stations_field, subroutes_fc):
    # Ensure necessary fields exist in subroutes feature class
    for field_name in ['upstream_station_id', 'downstream_station_id']:
        if field_name not in [f.name for f in arcpy.ListFields(subroutes_fc)]:
            arcpy.AddField_management(subroutes_fc, field_name, 'TEXT')

    # Iterate through each route in the routes feature class
    with arcpy.da.SearchCursor(routes_fc, ["route_id", sorted_stations_field, "SHAPE@"]) as routes_cursor:
        for route_id, sorted_station_ids, route_geom in routes_cursor:
            if sorted_station_ids:
                station_ids = sorted_station_ids.split(',')
                
                # Retrieve station geometries and distances
                station_distances = []
                for station_id in station_ids:
                    sql_query = f"STATION = {station_id}"
                    with arcpy.da.SearchCursor(stations_fc, ["SHAPE@", "STATION"], sql_query) as cursor:
                        for station_geom, station_id in cursor:
                            distance = route_geom.measureOnLine(station_geom.firstPoint)
                            station_distances.append((distance, station_id))

                # Sort distances along with station IDs
                station_distances.sort()

                # Create subroutes
                prev_distance = 0
                for i, (distance_to_station, upstream_station_id) in enumerate(station_distances):
                    downstream_station_id = station_distances[i + 1][1] if i + 1 < len(station_distances) else None

                    # Debugging print statements
                    print(f"Processing Route ID: {route_id}, Prev Distance: {prev_distance}, Distance to Station: {distance_to_station}")

                    # Check if the distance_to_station is within the route's length
                    if distance_to_station > route_geom.length:
                        print(f"Error: Station distance exceeds route length for route {route_id}, station {upstream_station_id}")
                        continue

                    subroute_geom = route_geom.segmentAlongLine(prev_distance, distance_to_station, use_percentage=False)

                    with arcpy.da.InsertCursor(subroutes_fc, ["SHAPE@", "route_id", "upstream_station_id", "downstream_station_id"]) as subroutes_cursor:
                        subroutes_cursor.insertRow([subroute_geom, route_id, upstream_station_id, downstream_station_id])

                    prev_distance = distance_to_station

                # Handle the last segment from the last station to the end of the route
                if station_distances:
                    last_station_id = station_distances[-1][1]
                    end_of_route = route_geom.length
                    print(f"Last segment for Route ID: {route_id}, Prev Distance: {prev_distance}, End of Route: {end_of_route}")
                    last_subroute = route_geom.segmentAlongLine(prev_distance, end_of_route, use_percentage=False)
                    with arcpy.da.InsertCursor(subroutes_fc, ["SHAPE@", "route_id", "upstream_station_id"]) as subroutes_cursor:
                        subroutes_cursor.insertRow([last_subroute, route_id, last_station_id])

# Call the function
create_subroutes_using_sorted_stations(routes_fc, stations_fc, 'sorted_stations', subroutes_fc)


Processing Route ID: 1, Prev Distance: 0, Distance to Station: 0.0
Processing Route ID: 1, Prev Distance: 0.0, Distance to Station: 31989.070018453614
Processing Route ID: 1, Prev Distance: 31989.070018453614, Distance to Station: 35321.236845183026
Processing Route ID: 1, Prev Distance: 35321.236845183026, Distance to Station: 77383.96950277835
Processing Route ID: 1, Prev Distance: 77383.96950277835, Distance to Station: 106903.70883196294
Processing Route ID: 1, Prev Distance: 106903.70883196294, Distance to Station: 162817.79676471054
Last segment for Route ID: 1, Prev Distance: 162817.79676471054, End of Route: 163189.51593795978
Processing Route ID: 2, Prev Distance: 0, Distance to Station: 0.0
Processing Route ID: 2, Prev Distance: 0.0, Distance to Station: 20957.657070683155
Processing Route ID: 2, Prev Distance: 20957.657070683155, Distance to Station: 50477.39639986775
Processing Route ID: 2, Prev Distance: 50477.39639986775, Distance to Station: 106391.48433261536
Last segme

RuntimeError: The coordinates or measures are out of bounds.

In [9]:
#change: let us also do a downstream station id here for eaach subroutes
def create_subroutes_using_sorted_stations(routes_fc, stations_fc, sorted_stations_field, subroutes_fc):
    # Ensure 'upstream_station_id' field exists in subroutes feature class
    if 'upstream_station_id' not in [f.name for f in arcpy.ListFields(subroutes_fc)]:
        arcpy.AddField_management(subroutes_fc, 'upstream_station_id', 'TEXT')

    # Iterate through each route in the routes feature class
    with arcpy.da.SearchCursor(routes_fc, ["route_id", sorted_stations_field, "SHAPE@"]) as routes_cursor:
        for route_id, sorted_station_ids, route_geom in routes_cursor:
            if sorted_station_ids:
                station_ids = sorted_station_ids.split(',')
                
                # Retrieve station geometries and distances
                station_distances = []
                for station_id in station_ids:
                    sql_query = f"STATION = {station_id}"
                    with arcpy.da.SearchCursor(stations_fc, ["SHAPE@", "STATION"], sql_query) as cursor:
                        for station_geom, upstream_station_id in cursor:
                            distance = route_geom.measureOnLine(station_geom.firstPoint)
                            station_distances.append((distance, upstream_station_id))

                # Sort distances along with station IDs
                station_distances.sort()

                # Create subroutes
                prev_distance = 0
                prev_station_id = None
                for distance_to_station, upstream_station_id in station_distances:
                    subroute_geom = route_geom.segmentAlongLine(prev_distance, distance_to_station, use_percentage=False)

                    with arcpy.da.InsertCursor(subroutes_fc, ["SHAPE@", "route_id", "upstream_station_id"]) as subroutes_cursor:
                        subroutes_cursor.insertRow([subroute_geom, route_id, prev_station_id])

                    prev_distance = distance_to_station
                    prev_station_id = upstream_station_id  # Set for the next subroute

                # Handle the last segment from the last station to the end of the route
                end_of_route = route_geom.length
                last_subroute = route_geom.segmentAlongLine(prev_distance, end_of_route, use_percentage=False)
                with arcpy.da.InsertCursor(subroutes_fc, ["SHAPE@", "route_id", "upstream_station_id"]) as subroutes_cursor:
                    subroutes_cursor.insertRow([last_subroute, route_id, prev_station_id])

# Call the function
create_subroutes_using_sorted_stations(routes_fc, stations_fc, 'sorted_stations', subroutes_fc)


RuntimeError: The coordinates or measures are out of bounds.

In [19]:
# old function
def create_subroutes_using_sorted_stations(routes_fc, stations_fc, sorted_stations_field, subroutes_fc):
    # Iterate through each route in the routes feature class
    with arcpy.da.SearchCursor(routes_fc, ["route_id", sorted_stations_field, "SHAPE@"]) as routes_cursor:
        for route_id, sorted_station_ids, route_geom in routes_cursor:
            if sorted_station_ids:
                # Split the sorted_station_ids into a list
                station_ids = sorted_station_ids.split(',')

                # Retrieve the geometries of the stations using SQL query
                station_distances = []
                for station_id in station_ids:
                    # Numeric field does not need single quotes in SQL query
                    sql_query = f"STATION = {station_id}"
                    with arcpy.da.SearchCursor(stations_fc, ["SHAPE@"], sql_query) as cursor:
                        for station_geom in cursor:
                            distance = route_geom.measureOnLine(station_geom[0].firstPoint)
                            station_distances.append(distance)

                # Sort the distances to ensure subroutes are created in the correct order
                station_distances.sort()

                # Create subroutes
                prev_distance = 0
                for distance_to_station in station_distances:
                    subroute_geom = route_geom.segmentAlongLine(prev_distance, distance_to_station, use_percentage=False)
                    with arcpy.da.InsertCursor(subroutes_fc, ["SHAPE@", "route_id"]) as subroutes_cursor:
                        subroutes_cursor.insertRow([subroute_geom, route_id])
                    prev_distance = distance_to_station

                # Handle the last segment from the last station to the end of the route
                end_of_route = route_geom.length
                last_subroute = route_geom.segmentAlongLine(prev_distance, end_of_route, use_percentage=False)
                with arcpy.da.InsertCursor(subroutes_fc, ["SHAPE@", "route_id"]) as subroutes_cursor:
                    subroutes_cursor.insertRow([last_subroute, route_id])

# Call the function
create_subroutes_using_sorted_stations(routes_fc, stations_fc, 'sorted_stations', subroutes_fc)


In [4]:
#delete subroutes_fc
arcpy.Delete_management(subroutes_fc)

<Result 'true'>

In [ ]:
'''archive
def update_routes_with_sorted_stations(routes_fc, station_route_ranks):
    # Name of the new field to store sorted station IDs
    sorted_stations_field = "sorted_stations"

    # Check if the new field exists; if not, create it
    field_names = [field.name for field in arcpy.ListFields(routes_fc)]
    if sorted_stations_field not in field_names:
        arcpy.AddField_management(routes_fc, sorted_stations_field, "TEXT", field_length=9999)

    # Create a temporary dictionary to store route to station mapping
    route_to_stations = defaultdict(list)

    # Populate the route_to_stations dictionary
    for station_id, routes in station_route_ranks.items():
        for route_id, rank in routes.items():
            route_to_stations[route_id].append((rank, station_id))
    
    # Update the routes feature class with sorted station lists
    with arcpy.da.UpdateCursor(routes_fc, ["route_id", "sorted_stations"]) as cursor:
        for row in cursor:
            route_id = row[0]
            if route_id in route_to_stations:
                # Sort stations by their rank and retrieve station IDs
                sorted_stations = [station_id for _, station_id in sorted(route_to_stations[route_id])]
                row[1] = ','.join(sorted_stations)  # Update with sorted station IDs
                cursor.updateRow(row)

# Call the function to update the routes feature class
update_routes_with_sorted_stations(routes_fc, station_route_ranks)
'''

Finally the rank is done. Below are subroutes splitting codes

In [1]:
# delete subroutes_fc if it exists
if arcpy.Exists(subroutes_fc):
    arcpy.Delete_management(subroutes_fc)

NameError: name 'arcpy' is not defined

In [5]:
# Output feature class for the sub-routes
subroutes_fc = 'subroutes_featureclass'

# Ensure the subroutes feature class exists
if not arcpy.Exists(subroutes_fc):
    arcpy.CreateFeatureclass_management(
        out_path=arcpy.env.workspace,
        out_name="subroutes_featureclass",
        geometry_type="POLYLINE",
        spatial_reference=arcpy.Describe(routes_fc).spatialReference)

route_field_names = [field.name for field in arcpy.ListFields(subroutes_fc)]

# Check if 'route_id' and 'route_rank' exist, create them if not
if 'route_id' not in route_field_names:
    arcpy.AddField_management(subroutes_fc, 'route_id', 'TEXT')

In [61]:
'''
def create_subroutes_using_sorted_stations(routes_fc, stations_fc, sorted_stations_field, subroutes_fc):
    # Check if 'route_id' exists in subroutes_fc, create it if not
    route_field_names = [field.name for field in arcpy.ListFields(subroutes_fc)]
    if 'route_id' not in route_field_names:
        arcpy.AddField_management(subroutes_fc, 'route_id', 'LONG')

    # Determine the data type of station_id
    station_id_field = arcpy.ListFields(stations_fc, "STATION")[0]
    is_station_id_numeric = station_id_field.type in ['Integer', 'SmallInteger', 'Float', 'Double']

    # Iterate through each route in the routes feature class
    with arcpy.da.SearchCursor(routes_fc, ["route_id", sorted_stations_field, "SHAPE@"]) as routes_cursor:
        for route_id, sorted_station_ids, route_geom in routes_cursor:
            if sorted_station_ids:
                station_ids = sorted_station_ids.split(',')

                # Format station IDs for SQL query
                formatted_station_ids = ["'{}'".format(station_id) if not is_station_id_numeric else station_id for station_id in station_ids]

                # Retrieve the geometries of the stations
                sql_query = "STATION IN ({})".format(','.join(formatted_station_ids))
                station_distances = [route_geom.measureOnLine(station_geom[0]) for station_geom in arcpy.da.SearchCursor(stations_fc, ["SHAPE@"], sql_query)]

                # Ensure distances are sorted
                station_distances.sort()

                # Initialize with the start of the route
                prev_distance = 0 

                # Iterate over each station distance, split the route, and create subroutes
                for distance_to_station in station_distances:
                    subroute_geom = route_geom.segmentAlongLine(prev_distance, distance_to_station, use_percentage=False)
                    
                            
                    # Add the subroute to the subroutes feature class
                    with arcpy.da.InsertCursor(subroutes_fc, ["SHAPE@", "route_id"]) as subroutes_cursor:
                        subroutes_cursor.insertRow([subroute_geom, route_id])
 

                    # Prepare for the next iteration
                    prev_distance = distance_to_station

                # Handle the last segment from the last station to the end of the route
                end_of_route = route_geom.length
                last_subroute = route_geom.segmentAlongLine(prev_distance, end_of_route, use_percentage=False)
                with arcpy.da.InsertCursor(subroutes_fc, ["SHAPE@", "route_id"]) as subroutes_cursor:
                    subroutes_cursor.insertRow([last_subroute, route_id])

# Call the function
create_subroutes_using_sorted_stations(routes_fc, stations_fc, 'sorted_stations', subroutes_fc)
'''

RuntimeError: The coordinates or measures are out of bounds.

In [5]:
'''#debugging second trial
def create_subroutes_using_sorted_stations(routes_fc, stations_fc, sorted_stations_field, subroutes_fc):
    # Check if 'route_id' exists in subroutes_fc, create it if not
    route_field_names = [field.name for field in arcpy.ListFields(subroutes_fc)]
    if 'route_id' not in route_field_names:
        arcpy.AddField_management(subroutes_fc, 'route_id', 'LONG')

    # Determine the data type of station_id
    station_id_field = arcpy.ListFields(stations_fc, "STATION")[0]
    is_station_id_numeric = station_id_field.type in ['Integer', 'SmallInteger', 'Float', 'Double']

    # Iterate through each route in the routes feature class
    with arcpy.da.SearchCursor(routes_fc, ["route_id", sorted_stations_field, "SHAPE@"]) as routes_cursor:
        for route_id, sorted_station_ids, route_geom in routes_cursor:
            if sorted_station_ids:
                station_ids = sorted_station_ids.split(',')

                # Format station IDs for SQL query
                formatted_station_ids = ["'{}'".format(station_id) if not is_station_id_numeric else station_id for station_id in station_ids]

                # Retrieve the geometries of the stations
                sql_query = "STATION IN ({})".format(','.join(formatted_station_ids))
                station_distances = [route_geom.measureOnLine(station_geom[0]) for station_geom in arcpy.da.SearchCursor(stations_fc, ["SHAPE@"], sql_query)]
                # Retrieve distances along the route
                station_distances = []
                for station_geom in arcpy.da.SearchCursor(stations_fc, ["SHAPE@"], sql_query):
                    distance = route_geom.measureOnLine(station_geom[0])
                    if 0 <= distance <= route_geom.length:
                        station_distances.append(distance)
                # Ensure distances are sorted
                station_distances = sorted(set(station_distances))

                # Initialize with the start of the route
                prev_distance = 0 

                # Iterate over each station distance, split the route, and create subroutes
                for distance_to_station in station_distances:
                    # Diagnostic print
                    print(f"Route ID: {route_id}, Segment: {prev_distance} to {distance_to_station}")

                    # Check if the segment distance is valid
                    if prev_distance != distance_to_station:
                        subroute_geom = route_geom.segmentAlongLine(prev_distance, distance_to_station, use_percentage=False)

                        # Add the subroute to the subroutes feature class
                        with arcpy.da.InsertCursor(subroutes_fc, ["SHAPE@", "route_id"]) as subroutes_cursor:
                            subroutes_cursor.insertRow([subroute_geom, route_id])
 

                    # Prepare for the next iteration
                    prev_distance = distance_to_station

                 # Handle the last segment from the last station to the end of the route
                end_of_route = route_geom.length
                if prev_distance != end_of_route:
                    # Diagnostic print
                    print(f"Route ID: {route_id}, Last Segment: {prev_distance} to {end_of_route}")

                    last_subroute = route_geom.segmentAlongLine(prev_distance, end_of_route, use_percentage=False)
                    with arcpy.da.InsertCursor(subroutes_fc, ["SHAPE@", "route_id"]) as subroutes_cursor:
                        subroutes_cursor.insertRow([last_subroute, route_id])

# Call the function
create_subroutes_using_sorted_stations(routes_fc, stations_fc, 'sorted_stations', subroutes_fc)'''


Route ID: 1, Segment: 0 to 0.0
Route ID: 1, Segment: 0.0 to 31989.070018453614
Route ID: 1, Segment: 31989.070018453614 to 35321.236845183026
Route ID: 1, Segment: 35321.236845183026 to 77383.96950277835
Route ID: 1, Segment: 77383.96950277835 to 106903.70883196294
Route ID: 1, Segment: 106903.70883196294 to 162817.79676471054
Route ID: 1, Last Segment: 162817.79676471054 to 163189.51593795978
Route ID: 2, Segment: 0 to 0.0
Route ID: 2, Segment: 0.0 to 20957.657070683155
Route ID: 2, Segment: 20957.657070683155 to 50477.39639986775
Route ID: 2, Segment: 50477.39639986775 to 106391.48433261536
Route ID: 2, Last Segment: 106391.48433261536 to 106763.20350586466
Route ID: 3, Segment: 0 to 0.0
Route ID: 3, Segment: 0.0 to 3666.817840090232
Route ID: 3, Segment: 3666.817840090232 to 57906.61636134438
Route ID: 3, Segment: 57906.61636134438 to 87426.35569052897
Route ID: 3, Segment: 87426.35569052897 to 143340.44362327657
Route ID: 3, Last Segment: 143340.44362327657 to 143712.16279652566
Ro

In [11]:
def delete_zero_length_subroutes(subroutes_fc):
    # Create a list to hold the IDs of subroutes to delete
    subroutes_to_delete = []

    # Use a search cursor to find subroutes with zero length
    with arcpy.da.SearchCursor(subroutes_fc, ["OID@", "Shape_Length"]) as cursor:
        for row in cursor:
            if row[1] == 0:
                subroutes_to_delete.append(row[0])

    # Check if there are any subroutes to delete
    if subroutes_to_delete:
        # Use an update cursor to delete the zero-length subroutes
        with arcpy.da.UpdateCursor(subroutes_fc, ["OID@"]) as cursor:
            for row in cursor:
                if row[0] in subroutes_to_delete:
                    cursor.deleteRow()

# Call the function
delete_zero_length_subroutes(subroutes_fc)
